# Endimensjonal kvantemekanisk brønnmodell for atomer, molekyler og krystaller

I denne notatboken skal vi undersøke en enkel kvantemekanisk modell for atomer, molekyler og krystaller der potensialet omkring enkeltatomer modelleres med brønnpotensialer.

Omkring en atomkjerne vil det være et potensiale som elektroner med masse $m_e$ kan befinne seg i.
I vår enkle, endimensjonale modell vil vi beskrive dette potensialet som et brønnpotensial

$$V(x) = \begin{cases} -V_0 & |x| \leq w / 2 \\ 0 & \text{ellers} \\ \end{cases}$$

omkring et atom med bredde $w$ sentrert i origo.
For å beskrive molekyler og atomer, konstruerer vi nye potensialer som består av like mange slike brønnpotensialer som det er atomer i molekylet eller krystallen, plassert etter hverandre og separert med områder med null potensiale og bredde $b$ som modellerer bindinger og bindingslengder i strukturen.

Vi neglisjerer vekselvirkning mellom elektroner rundt atomkjernen, slik at hvert elektron befinner seg i en stasjonær tilstand $\Psi(x, t) = \psi(x) e^{i E t / \hbar}$ med tilhørende energi $E$, der energiegenfunksjonen $\psi(x)$ løser den tidsuavhengige Schrödingerligningen

$$-\frac{\hbar^2}{2 m_e} \psi''(x) + V(x) \psi(x) = E \psi(x)$$

der $V(x)$ er et (annet) potensial og $\hbar$ er Plancks konstant.

Elektronet er et fermion som kan ha to spinnverdier, og Paulis eksklusjonsprinsipp forteller oss da at maksimalt to elektroner vil okkupere hver stasjonære tilstand.

Schrödingerligningen har sjelden analytiske løsninger.
Derfor vil vi her løse ligningen numerisk ved å hakke opp rommet i $N + 2$ punkter $x_0 = 0, x_1, \ldots, x_N, x_{N+1} = L$ med avstand $\Delta x = L / (N + 1)$ mellom hvert punkt.
Hvert punkt tilordnes potensialet $V_n = V(x_n)$ og verdien $\psi_n = \psi(x_n)$ for energiegenfunksjonen.
For å avgrense problemet, innfører vi randbetingelsene $V_0 = V_{N+1} = \infty$, slik at $\psi_0 = \psi_{N+1} = 0$.
I tillegg tilnærmer vi de deriverte av energiegenfunksjonen med
$$\psi_n' = \psi'(x_n) = \frac{\psi_{n+1} - \psi_n}{\Delta x} \qquad \text{og} \qquad \psi_n'' = \psi''(x_n) = \frac{\psi_n' - \psi_{n-1}'}{\Delta x} = \frac{\psi_{n+1} - 2 \psi_n + \psi_{n-1}}{{\Delta x}^2}$$

Problemet er dermed redusert til å finne verdier $\psi_1, \ldots, \psi_N$ for energiegenfunksjonen og tilhørende energier $E$ som oppfyller
$$-\frac{\hbar^2}{2 m_e} \frac{\psi_{n+1} - 2 \psi_n + \psi_{n-1}}{{\Delta x}^2} + V_n \psi_n = E \psi_n$$
Dette er ekvivalent med å finne egenvektorer $\boldsymbol{\psi} = [\psi_1, \ldots, \psi_N]^T$ og tilhørende egenverdier $E$ som oppfyller matriseligningen
$$H \boldsymbol{\psi} = E \boldsymbol{\psi}$$
der matrisen $H$ er en symmetrisk matrise med $H_{n, n} = \hbar^2 / m {\Delta x}^2 + V_n$ langs diagonalen, $H_{n, n \pm 1} = \hbar^2 / 2 m {\Delta x}^2$ over og under diagonalen og $0$ alle andre steder.
Dette er et velkjent problem fra lineær algebra med mange effektive programvareimplementasjoner, og er utgangspunktet for vår numeriske løsningsmetode.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## Setup plotting parameters
newparams = {'axes.labelsize': 15,
             'axes.linewidth': 1,
             'lines.linewidth': 1.5, 
             'figure.figsize': (8, 5),
             'ytick.labelsize': 15,
             'xtick.labelsize': 15,
             'ytick.major.pad': 5,
             'xtick.major.pad': 5,
             'legend.fontsize': 15,
             'legend.frameon': True, 
             'legend.handlelength': 1.5,
             'axes.titlesize': 20,
             'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

In [ ]:
WAVE_SCALE_FACTOR = 10 # Factor to scale wave with, so that when plotting we get
                        # y = engy + WAVE_SCALE_FACTOR * wave
hbar = 1.05e-25 # nJs
m = 9.11e-31 # kg
eV = 1.60e-19 # J

try:
    from numpy import heaviside
    heaviside = heaviside
except:
    heaviside = lambda x,a: (x > 0) * 1

# Some does not have scipy.linalg.eigh_tridiagonal, so this takes care of that
# TODO: leave only one in the end
try:
    from scipy.linalg import eigh_tridiagonalu
    get_eigh = lambda H: eigh_tridiagonal(np.diag(H), np.diag(H, k=1))
except ImportError:
    get_eigh = np.linalg.eigh
    
def V_crystal(x, w, b, L, V0):
    n = (x - 10 * w) / (w + b)
    V = np.where((x > 10 * w) & (x < L - 10 * w + b) & ((n - np.floor(n)) * (w + b) < w), -V0, 0)
    return V

def get_space_length(N_w, w, b):
    return 10 * w + N_w * w + (N_w - 1) * b + 10 * w

def partition_space(N, N_w, w, b):
    L = get_space_length(N_w, w, b)
    dx = L / (N + 1)
    x = np.linspace(dx, N * dx, N)
    return L, dx, x

def get_stationary_states(dx, V):
    factor = hbar**2 / (m * dx**2) / eV
    diag = factor + V
    second_diag = -factor/2 * np.ones(diag.shape[0] - 1)
    H = np.diag(diag) + np.diag(second_diag, k=1) + np.diag(second_diag, k=-1)
    
    eigvals, eigvecs = get_eigh(H)
    energies, waves = eigvals, eigvecs.T
    return energies, waves

def plot_potential(x, V, **kwargs):
    plt.plot(x, V, color=kwargs.pop("potential_color", "grey"),
                linestyle=kwargs.pop("potential_style", "solid"),
                linewidth=kwargs.pop("potential_linewidth", 3),
                label="$V(x)$")

def plot_stationary_states(x, energies, waves,
                           include_engy_legend = True,
                           clip_y_axis = False, **kwargs):
    wave_scale_factor = kwargs.pop("wave_scale_factor", WAVE_SCALE_FACTOR) # The amplitude of waves is not really interesting, just scale so that they are visible in the plot
    
    for n, (energy, wave) in enumerate(zip(energies, waves)):
        color = "C%d" % (n % 10)
        label = "$E_{%d} = %+.1f \,\, eV, \psi_{%d}$" % (n, energy, n) if include_engy_legend else None
        plt.plot((0, x[-1]), (energy, energy), color=color, linestyle="dashed", label=label)
        
        plt.plot(x, energy + wave * wave_scale_factor, color=color, linestyle="solid")
        
    if clip_y_axis:
        # Sets upper y limit to biggest wave
        y_max = np.amax( wave_scale_factor * waves + energies[:, np.newaxis] )   
        
        axes = plt.gca()
        axes.set_ylim([None, y_max + 0.1 * np.abs(y_max)])
        
    plt.legend()
    
def investigate_potential(N, N_w, w, b, n, V0,
                          show_legend = True,
                          return_energies = False,
                          **kwargs):
    """
    :include_engy_legend : whether to include labels for energy in legend
    :clip_y_axis         : sets upper limit of y_axis to biggest wave value,
                           so that if potential is much bigger than wave, we can "zoom-in"
    """
    L, dx, x = partition_space(N, N_w, w, b)
    V = V_crystal(x, w, b, L, V0)
    energies, waves = get_stationary_states(dx, V)
    
    print("Greatest deviation from orthonormality:")
    print(np.max(np.abs(waves @ waves.T - np.eye(N))))

    plt.figure(figsize=(16, 8))
    plt.title("%d lowest-energy states with %d wells, well width %.2f and well distance %.2f" % (n, N_w, w, b))
    plt.xlabel("$x \,\, (nm)$")
    plt.ylabel("$V(x), E_n \,\, (eV)$")
    plot_potential(x, V, **kwargs)
    plot_stationary_states(x, energies[:n], waves[:n], **kwargs) # Pass **kwargs to gain better control over plotting
    
    plt.legend(loc="upper left")
    
    if return_energies:
        return energies

N = 400
w = 0.5 # nm
c = 8 # integer
b = w / c # nm
n = 6
V0 = 10


investigate_potential(N, 0, w, b, n, V0)
investigate_potential(N, 1, w, b, n, V0)
investigate_potential(N, 2, w, b, n, V0)
investigate_potential(N, 10, w, b, n, V0, clip_y_axis = True)


Koden nedenfor beregner energinivåene i en uendelig dyp potensialbrønn.

In [ ]:
def get_energy_level(j, width = w, mass = m):
    return hbar**2 * j * np.pi / (width * 2 * mass * eV) 

for i in range(1, n):
    print('E_', i, ' = ', f'{get_energy_level(i):.4f}', ' eV', sep='')

# Oppgave 2

### a)
Figuren viser en potensialbrønn med fire tilhørende bølgefunksjoner. 
Brønnens dybde er her valgt slik at det finnes tre bølgefunksjoner innenfor brønnen, mens dybden er satt til $0.1 \, nm $ -- omtrent tilsvarende et atoms radius.
Dette for å oppnå tall av en mer naturtro størrelsesorden. 

Fra figuren kommer det frem at annenhver bølgeunksjon er symmetrisk og antisymmetrisk, hvilket stemmer med teori. 
Man ser også at bølgefunksjonene for grunntilstand, første og andre eksiterte tilstand henholdsvis har 0, 1 og 2 nullpunkter. 
De oppfyller også initialbetingelsene om at de skal være kontinuerlige og at de går mot null i stor avstand fra brønnen, når potensialet er null.

Denne potensialfunksjonen kan modellere et atom, og bølgefunksjonene er da mulige tilstander atomets elektroner kan befinne seg i. 
Energien til disse tilstandene er gitt ved likningene 

$\sqrt{ \frac{2 m_e V_0 l^2}{\hbar^2} - (kl)^2 } = kl \tan{kl}$,

for symmetriske, og 

$\sqrt{ \frac{2 m_e V_0 l^2}{\hbar^2} - (kl)^2 } = kl \tan{kl - 
\frac{\pi}{2}}$,

for antisymmetriske energiegenfinksjoner, der

$kl = \frac{l}{\hbar} \sqrt{2 m_e E}$,

$E_j = \frac{\hbar^2 k_j^2}{2m}, \,\, j = 1,2,\dots,$

og $E$ er energiegenverdien, og $m$ er massen til partikkelen i brønnen (som her er elektronmassen). 

$k_j = j \pi / l$ er bølgetallet til egenfunksjonene, med $l$ lik brønnens bredde.

For denne potensialbrønnen er disse analytiske energiegenverdiene som følger:

In [ ]:
def left_side(kl, V_0 = V0, l = w):
    if ((2*m*V_0*l**2 * eV)/(hbar**2)) - kl**2 > 0:
        return np.sqrt(((2*m*V_0*l**2 * eV)/(hbar**2)) - kl**2)
    else:
        return np.nan

def kappa_l(kl, sym=1):
    if sym:
        return kl * np.tan(kl)
    else:
        return kl * np.tan(kl - np.pi/2)

def right_side(kl, lower = 0, upper = 10, l=w):
    if kappa_l(kl) > lower and kappa_l(kl, 0) < upper:
        return kappa_l(kl)
    elif kappa_l(kl, 0) > lower and kappa_l(kl, 0):
        return kappa_l(kl, 0)
    else:
        return np.nan
    
def get_analytic_solutions(tol, left, right, steps, V_0 = V0):
    kl = 0
    solutions = [[], []]
    diff = [left[i] - right[i] for i in range(len(left))]
    for i in range(len(diff)):
        if abs(diff[i]) < tol:
            solutions[0].append(i)
            solutions[1].append(right[i])
    
    print('Approximate analytic solutions for k*l:') 
    kl_radians = []
    for i in range(len(solutions[0])):
        if float(f'{solutions[0][i]*4/steps:.2f}') not in kl_radians:
            print(f'{solutions[0][i]*4/steps:.2f}', 'pi')
            kl_radians.append(float(f'{solutions[0][i]*4/steps:.2f}'))
    return solutions, kl_radians

def kl_to_energy(kl, l = w, V_0 = V0):
    E = []
    for el in kl:
        E.append(-float(f'{(el**2 * hbar**2)/(2*m*l**2*eV):.2f}'))
        # Something is very wrong with the energy values
    return E

def right_minus_left(kl, V_0=V0):
    return right_side(kl) - left_side(kl)

def plot_analytic_solutions(steps, V_0 = V0):
    left = []
    right = []
    for i in range(0,steps):
        kl = i*np.pi*4/steps
        left.append(left_side(kl))
        right.append(right_side(kl))
        
    for i in range(len(right)):
        if right[i] > 9:
            right[i] = np.nan
                    
    solutions, radian_solutions = get_analytic_solutions(0.01, left, right, steps)
    
    xvals = [steps/8, steps/4, 3*steps/8, steps/2, 5*steps/8]
    pifracs = ['$\pi/2$', '$\pi$', '$3\pi/2$', '$2\pi$', '$5\pi/2$']
    
    plt.figure(figsize=(16, 8))           
    plt.xticks(xvals,pifracs)
    plt.plot(left)
    plt.plot(right)
    plt.plot(solutions[0], solutions[1], 'ro')
    plt.ylim(0,left[0] + 0.5)
    plt.xlim(0,0.7*steps)
    plt.xlabel('$k\ell = \ell / \hbar \sqrt{2mE}$')
    plt.ylabel('$\kappa \ell$')
    plt.show()
    print('Approximate analytic solutions for the energy levels:')
    for i in range(len(radian_solutions)):
        print('E_', i, ': ', kl_to_energy(radian_solutions)[i], ' eV', sep='')
    
plot_analytic_solutions(20000)

Det er tegnet inn en bølgefunksjon ovenfor brønnen, i tillegg til de tre i brønnen. 
Denne representerer bølgefunksjonen for et ubundet elektron (fritt elektron), som har for stor kinetisk energi til å fanges av det elektriske feltet til atomkjernen. 
Et slikt elektron skal, langt unna atomet, ha en upåvirket, sinusoidal bølgefunksjon. 
Bølgelengden til elektronet leses fra grafen av til å være omkring $1 \, nm$.
### TODO Les av bølgelengden. Si litt mer om elektronet?

### b)
Med tre bundne tilstander er det plass til inntil seks elektroner innenfor potensialbrønnen. 
Dette fordi Pauliprinsippet utelukker muligheten for at det kan eksistere mer enn to elektroner (ett med spin opp og ett med spinn ned) i hver bundne tilstand. 
En slik potensialbrønn kan modellere atomer av nummer én til og med seks (hydrogen til og med karbon), som har ett til seks elektroner i grunntilstand. 
Disse atomenes totale elektronspinn finnes fra Pauliprinsippet. 
Dersom et atom har et partall antall elektroner (slik som i helium og karbon), vil dets totale elektronpinn i grunntilstand være null. 
Dersom det har et odde antall elektroner (f.eks. hydrogen, litium), blir spinnet $S = \pm \frac{\sqrt{3}}{2} \hbar$. 
I eksiterte tilstander kan spinnet også ta andre verdier, alle heltallige multiplum av $\pm \frac{\sqrt{3}}{2} \hbar$.

# Oppgave 3

Dobbel potensialbrønn, hver med samme dybde og lengde som i oppgave 2.
Kan modellere et molekyl.
Får dobbelt så mange bundne stasjonære tilstander som i oppgave 2, dvs. totalt 6.

Ser av figuren at to separate H-atomer til sammen har energien $2 \cdot (-179.0 \, eV) = -358.0 \, eV$ i grunntilstanden.
Et H2-molekyl har energien $2 \cdot (-179.6 \, eV) = -359.2 \, eV$ i grunntilstanden.
H2-molekylet har dermed en bindingsenergi $(-359.2 \, eV) - (-358.0 \, eV) = -1.2 \, eV$ relativt de separate H-atomene.
Den negative differansen indikerer at H2-molekylet er mer stabilt enn de separate H-atomene, noe som stemmer med erfaring.

To separate He-atomer har til sammen energien $2 \cdot 2 \cdot (-179 \, eV) = -716.0 \, eV$.
Et He2-molekyl har energien $2 \cdot (-179.6 \, eV) + 2 \cdot (-175.5 \, eV) = -710.2 \, eV$.
He2-molekylet har dermed en bindingsenergi $(-710.2 \, eV) - (-716.0 \, eV) = + 5.8 \, eV$ relativt de separate He-atomene.
Den positive differansen indikerer at He2-molekylet er mindre stabilt enn de separate He-atomene, noe som stemmer med erfaring, da He er en edelgass.

Disse resultatene kan knyttes opp mot teori om bindende og antibindende orbitaler.
I H2-molekylet okkuperer begge elektronene en energitilstand med noe lavere energi enn grunntilstanden for et H-atom.
En slik energitilstand gjør molekylet mer stabilt enn de separate atomene, og kalles derfor en bindende orbital.
I He2-molekylet okkuperer to av elektronene en energitilstand med noe lavere energi enn grunntilstanden for et He-atom, og to av elektronene en energitilstand med noe høyere energi enn grunntilstanden for et He-atom.
Til sammen får molekylet en høyere energi enn de separate atomene og er derfor minst stabilt, og den høyeste energitilstanden av de to i molekylet kalles derfor en antibindende orbital, på grunn av denne effekten.

<!-- TODO: sammenlign kvantitativt (i hvert fall relativt mellom H og He) fra tall i Wikipedia-artikler -->
<!-- https://en.wikipedia.org/wiki/Helium_dimer -->
<!-- https://en.wikipedia.org/wiki/Bond-dissociation_energy -->
<!-- https://en.wikipedia.org/wiki/Antibonding_molecular_orbital -->

# Om nøyaktigheten til modellen

Det er viktig å understreke at brønnmodellen er en primitiv modell for atomer, molekyler og krystaller.
Den vil ikke gi nøyaktige eller kvantitativt korrekte tallverdier, men den lar oss belyse enkelte kvalitative egenskaper.
For eksempel vil vi her benytte en brønn med en fast verdi for brønndybden $V_0$ til å modellere ulike atomer med ulike antall protoner i kjernen, selv om brønndybden rundt ulike atomer med forskjellige antall protoner i kjernen i realiteten vil variere.
For å ha et rimelig bilde av situasjonene vi modellerer, vil vi regne med korrekte verdier for Plancks (reduserte) konstant og elektronmassen og en rimelig verdi for brønnbredden, som i denne modellen tilsvarer atombredden på omtrent 0.1 nm.
For å tvinge frem observerbare forskjeller på de ulike situasjonene og illustrere kvalitative forskjeller, er vi derimot nødt til å redusere bindingslengden mellom atomer noe under dens realistiske verdi, og til slutt brønndybden til å gi et ønsket antall bundne tilstander.

# Oppgave 4 - Krystaller

## a)
Vi plotter energinivåene og egenfunksjonene for henholdsvis $N_w = 5$ og $N_w = 10$.

In [ ]:
# Find atomic energies
L, dx, x = partition_space(N, 1, w, b)
V = V_crystal(x, w, b, L, V0)
energies, waves = get_stationary_states(dx, V)

# Investigate
num_wells = 5
num_engy_to_show = 2 * num_wells + 1
investigate_potential(N, num_wells, w, b, num_engy_to_show, V0,
                      include_engy_legend = False,
                      clip_y_axis = True,
                      potential_style = "dashed",
                      potential_width = 2)

num_wells = 10
num_engy_to_show = 2 * num_wells + 1
investigate_potential(N, num_wells, w, b, num_engy_to_show, V0,
                      include_engy_legend = False,
                      clip_y_axis = True,
                      potential_style = "dashed",
                      potential_width = 2)

Vi observerer at med $N_w = 5$ og $N_w = 10$ får vi henholdsvis 5 og 10 energinivåer i "bånd" rundt de to første energinivåene i et atom, slik to atomer fikk to bundne tilstander rundt enkeltatomets energinivåer.

## b)
Når vi skal beregne båndbredden for krystaller av et stort antall atomer observerer vi at det blir numeriske feil. Disse manifesterer seg ved at båndbredde som funksjon av antall brønner blir lite glatt. For å unngå dette benytter vi her en annen metode for å generere potensialet.

Feilen med forrige potensialfunksjon kommer av at det er avrunding som bestemmer om et punkt på $x$-aksen er i en brønn eller på en barriære. Dette har mye å si når bredden på en barriære er få punkter.

Den nye metoden for å generere potensialet, `V_crystal_2`, tar et annet utgangspunkt enn forrige metode. I stedenfor å partisjonere $x$-intervallet i $N$ punkter, for så å finne ut hvilken verdi potensialet har i hvert punkt, tar den nye metoden utgangspunkt i at hver brønn og barriære er et gitt antall punkter bred.

In [ ]:
def V_crystal_2(N_w, n, n_b, V0):
    """Generate potential with N_w wells using
    n as points in a well and n_b as points in a barrier
    
    Input
    :N_w  : number of wells
    :n    : points in a well
    :n_b  : points in a barrier
    :V0   : depth of well
    
    Output
    :potential : a list of points, with length 
                 N = 2 * 10 * n + N_w * (n + n_b)
    """
    
    well = np.append(-V0*np.ones(n), np.zeros(n_b)) # One well plus barrier
    wells = np.tile(well, N_w)
    potential = np.append(np.zeros(10*n), wells) # Pad 10w before wells
    potential = np.append(potential, np.zeros(10*n - n_b)) # Pad after wells
    return potential
    

In [ ]:
# Set up some useful values
n_b = 3   # Points per barrier
n   = c * (n_b - 1) + 1  # Points per well
dx = b / (n_b - 1)  # Distance between each point

### Plot båndbredde til første 3 bånd som funksjon av antall brønner

In [ ]:
## Find bandwidth of three first bands as function of number of wells
to_wells = 100 # Max number of wells to check

band_widths = np.empty((3,0)) # We must have an empty list to use append

for num in range(1, to_wells + 1):
    V = V_crystal_2(num,n,n_b, V0)
    energies, waves = get_stationary_states(dx, V)
    bonds = energies[:3*num].reshape(-1, num)[:, [0, -1]] # Three first bonds, first and last value
    band_widths = np.append(band_widths, np.diff(bonds), axis=1)

In [ ]:
## Plot bandwidth
plt.title("Båndbredde som funksjon av antall brønner")
plt.xlabel("Antall brønner")
plt.ylabel("Båndbredde ($eV$)")

for i, band in enumerate(band_widths):
    plt.plot(np.arange(1,len(band) + 1), band, label="Bånd " + str(i + 1))
    plt.axhline(band[-1], linewidth=1, color="grey", alpha=.8, linestyle="dashed")
    
plt.legend(bbox_to_anchor=(0.97, 0.9))

Vi ser av figuren at alle de tre båndene raskt konvergerer mot en fast bredde.

## c)

In [ ]:
## Plot the first three bonds with num_wells wellso
num_wells = 10
num_to_plot = 3 * num_wells + 1  # Number of energies to plot.
                                 # Since each band consists of num_wells energies,
                                 # this gives the first three bands plus one energy level
        
# Find length of entire system
# TODO : calculate length of barrier explicitly, and rewrite this line
L = get_space_length(num_wells, w, b)

V = V_crystal_2(num_wells, n, n_b, V0)
energies, waves = get_stationary_states(dx, V)

x = np.linspace(0, L, len(V)) # Generate some points to use as x-axis

plot_potential(x, V)
plot_stationary_states(x, energies[:num_to_plot], waves[:num_to_plot], wave_scale_factor = 10,
                                                                       include_engy_legend = False)

# Todo: Skjekke at disse tallene også gjelder før vi leverer

At de $4N_w$ elektronene okkuperer de $2N_w$ første orbitalene tilsvarer at de to første båndene er fyllt opp og at det tredje båndet er helt tomt. De (to) elektronene med høyest energi er derfor elektronene som befinner seg i høyeste energi i andre bånd, som vi ut fra figuren leser til å være ca. $-6 \mathrm{eV}$.

Nærmeste ledige tilstand er da første energinivå i tredje bånd, som har en verdi som vi fra figuren ser at er ca. $-3\mathrm{eV}$. _Båndgapet_ er altså ca. $3 \mathrm{eV}$3. Dette er på grensen mellom halvleder og isolator.

## d)


In [ ]:
plot_potential(x, V, potential_linewidth = 1)
plot_stationary_states(x, energies[:3], waves[:3], wave_scale_factor = 10,
                                                             include_engy_legend = False,
                                                             clip_y_axis = True)

Fra figuren ser man tydelig at bølgefunksjonene er modulert med både en funksjon $u$ slik at $u(x) = u(x + a)$ der $a$ er krystallens periodisitet og en $sin$-funksjon med bølgetall $k$ som er elektronets bølgetall.

Legg merke til at de tre funksjonene plottet her har "hovedbølgelengde" lik to, en og $\frac32$ ganger lengden av brønnområdet.